'''
THIS FILE IS FOR TRAINING THE TREE PHENOLOGY MODEL
CODE WRITTEN FOR CUBESAT MANTIS PROGRAM (DSS)
AUTHOR: CHAZ DAVIES
''' 


In [2]:

import tensorflow as tf
import numpy as np
import matplotlib as img 
import rasterio
import os
import image_process as ip
from rasterio.plot import show
import json
import keras


In [3]:

path = 'C:\\Users\\chazd\\OneDrive\Desktop\\git\DSS\DSS-MANTIS-OBC-AI\\Images\\tree-phen\\trees_feb2020-Dec2021_10m'

In [4]:
## set constants
# might change during training process
image_height = 160
image_width = 175
bands = 4


In [5]:
# Make array to store label names

labels = {
        "evergreen-coverage" : 0, # when deciduous trees had no leaves
        "light-canopy" : 1, # when leaves are starting to form
        "dense-canopy" : 2, # when the tresses are at their greenest
        "start-of-senescence" : 3, # when leaves begin to change colour,
        "senescence" : 4 # when the leaves are very colourful
    }


labels

{'evergreen-coverage': 0,
 'light-canopy': 1,
 'dense-canopy': 2,
 'start-of-senescence': 3,
 'senescence': 4}

In [6]:
# copy the path
tiff_files = ip.loadDataSet(path)
file_path = ["" for x in range(len(tiff_files))]

# concactinate the folder path to the file name
for i in range(len(tiff_files)):
    file_path[i] = os.path.join(path, tiff_files[i])


In [7]:
#extract dates
dates = ["" for x in range(len(file_path))]
for i, file in enumerate(file_path):
    dates[i] = file[len(file)-15:len(file)-5]



In [8]:
raster_imgs = np.empty(len(file_path)) 

#open each file
img = rasterio.open(file_path[1])

    


c:\Users\chazd\AppData\Local\Programs\Python\Python39\lib\site-packages\rasterio\__init__.py:331: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [9]:
# Open images and move them to a numpy array
batch = len(file_path)
# set dimension of 3d-array to image height, width and number of bands
# ---- maybe add a dimension for labels

unlabeled_images = np.zeros((batch, image_height, image_width, bands))

for i in range(batch):
    #open raster image
    img = rasterio.open(file_path[i])

    for j in range(bands):
        unlabeled_images[i,:,:,j] = img.read(j+1)



In [10]:
# Load JSON file with image file names and their corresponding labels
with open('C:\\Users\\chazd\\OneDrive\\Desktop\\git\\DSS\\DSS-MANTIS-OBC-AI\\Images\\trees_feb2020-Dec2021_10m_labels.json') as json_file:
    data = json.load(json_file)

# Extract filenames and annotations
file_names = [item["filename"] for item in data]
annotations_from_json = [item["annotations"] for item in data]

# Map annotations from JSON to numerical labels using the label dictionary
numerical_labels = np.array([labels[annotation] for annotation in annotations_from_json])


## Preprocessing

Before prepocessing the images, a quick review of the dataset

In [11]:
print("Dimensions: ", unlabeled_images.shape)
print("Batch Size: ", len(unlabeled_images))

Dimensions:  (66, 160, 175, 4)
Batch Size:  66


In [12]:
# normalize all values
import cv2
import numpy as np

def normalize_4channel_image(image):
    # Split the image into Red, Green, Blue, and NIR channels
    r, g, b, nir = cv2.split(image)

    # Normalize each channel
    r_normalized = (r - np.min(r)) / (np.max(r) - np.min(r))
    g_normalized = (g - np.min(g)) / (np.max(g) - np.min(g))
    b_normalized = (b - np.min(b)) / (np.max(b) - np.min(b))
    nir_normalized = (nir - np.min(nir)) / (np.max(nir) - np.min(nir))

    # Recombine the channels
    normalized_image = cv2.merge([r_normalized, g_normalized, b_normalized, nir_normalized])

    return normalized_image



In [13]:
normalized= unlabeled_images

for i in range(len(unlabeled_images)):
    
    normalized[i] = normalize_4channel_image(unlabeled_images[i])
    


In [14]:
#split into test and train
split = np.random.randint(0,len(normalized), size = int(len(normalized)*0.9))

train_images = normalized[split]
train_labels = numerical_labels[split]
test_images = normalized[-split]
test_labels = numerical_labels[-split]


## Model First Itertion

In [15]:
from tensorflow.keras import datasets, layers, models

In [16]:
model = models.Sequential()

model.add(layers.Conv2D(16,(3,3),1,activation= 'softmax', input_shape=(160,175,4)))
model.add(layers.MaxPooling2D())

model.add(layers.Conv2D(32,(3,3),1, activation= 'softmax'))
model.add(layers.MaxPooling2D())

model.add(layers.Conv2D(16,(3,3),1, activation= 'softmax'))
model.add(layers.MaxPooling2D())

# ry softmax

model.add(layers.Flatten())
model.add(layers.Dense(16, activation='softmax'))
model.add(layers.Dense(5, activation='softmax')) 

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 158, 173, 16)      592       
                                                                 
 max_pooling2d (MaxPooling2  (None, 79, 86, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 77, 84, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 38, 42, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 36, 40, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 18, 20, 16)       

In [17]:
#compile
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))


test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Accuracy = ", test_acc*100, "%")


## VGG16 Model

In [42]:
# Remove Blue band
X_train = np.delete(train_images, 2, 3)
X_test = np.delete(test_images, 2, 3)
y_train = train_labels
y_test = test_labels

In [27]:
width = train_images.shape[1]
height = train_images.shape[2]
bands = 3

vgg16 = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape= (width, height, bands),
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)

In [29]:
vgg16.trainable = False


inputs = keras.Input(shape=(width, height, bands))
x = vgg16(inputs, training=False)

x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(5)(x)
vgg16_model = keras.Model(inputs, outputs)

In [31]:
vgg16_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 160, 175, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 5, 5, 512)         14714688  
                                                                 
 global_average_pooling2d_1  (None, 512)               0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_3 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 14717253 (56.14 MB)
Trainable params: 2565 (10.02 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [43]:
vgg16_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

vgg16_model.fit(X_train, y_train, epochs=11, 
                    validation_data=(X_test, y_test))

Epoch 1/11


2/2 [==============================] - 11s 7s/step - loss: 2.0174 - accuracy: 0.1017 - val_loss: 1.8431 - val_accuracy: 0.1356
Epoch 2/11
2/2 [==============================] - 8s 6s/step - loss: 1.8756 - accuracy: 0.1017 - val_loss: 1.7162 - val_accuracy: 0.1356
Epoch 3/11
2/2 [==============================] - 8s 6s/step - loss: 1.7540 - accuracy: 0.1695 - val_loss: 1.6034 - val_accuracy: 0.2373
Epoch 4/11
2/2 [==============================] - 9s 7s/step - loss: 1.6502 - accuracy: 0.2542 - val_loss: 1.5067 - val_accuracy: 0.5424
Epoch 5/11
2/2 [==============================] - 10s 7s/step - loss: 1.5538 - accuracy: 0.5085 - val_loss: 1.4265 - val_accuracy: 0.6102
Epoch 6/11
2/2 [==============================] - 10s 7s/step - loss: 1.4859 - accuracy: 0.5254 - val_loss: 1.3607 - val_accuracy: 0.6102
Epoch 7/11
2/2 [==============================] - 9s 7s/step - loss: 1.4201 - accuracy: 0.5254 - val_loss: 1.3100 - val_accuracy: 0.6102
Epoch 8/11
2/2 [====================

In [46]:
test_loss, test_acc = vgg16_model.evaluate(X_test,  y_test, verbose=2)
print("Accuracy = ", test_acc*100, "%")

2/2 - 5s - loss: 1.2106 - accuracy: 0.6102 - 5s/epoch - 3s/step
Accuracy =  61.01694703102112 %


### Results

## Resnet

In [47]:
width = train_images.shape[1]
height = train_images.shape[2]
bands = 3

resnet = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(width, height, bands),
    pooling=None,
)

94765736/94765736 [==============================] - 160s 2us/step


In [48]:
resnet.trainable = False


inputs = keras.Input(shape=(width, height, bands))
x = resnet(inputs, training=False)

x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(5)(x)
resnet_model = keras.Model(inputs, outputs)

In [49]:
resnet_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 160, 175, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 5, 6, 2048)        23587712  
                                                                 
 global_average_pooling2d_2  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_4 (Dense)             (None, 5)                 10245     
                                                                 
Total params: 23597957 (90.02 MB)
Trainable params: 10245 (40.02 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [50]:
resnet_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

resnet_model.fit(X_train, y_train, epochs=11, 
                    validation_data=(X_test, y_test))

Epoch 1/11
2/2 [==============================] - 11s 7s/step - loss: 1.2862 - accuracy: 0.5254 - val_loss: 1.1969 - val_accuracy: 0.6102
Epoch 2/11
2/2 [==============================] - 9s 7s/step - loss: 1.2674 - accuracy: 0.5254 - val_loss: 1.1925 - val_accuracy: 0.6102
Epoch 3/11
2/2 [==============================] - 9s 7s/step - loss: 1.2523 - accuracy: 0.5254 - val_loss: 1.1893 - val_accuracy: 0.6102
Epoch 4/11
2/2 [==============================] - 13s 10s/step - loss: 1.2371 - accuracy: 0.5254 - val_loss: 1.1829 - val_accuracy: 0.6102
Epoch 5/11
2/2 [==============================] - 14s 10s/step - loss: 1.2252 - accuracy: 0.5254 - val_loss: 1.1765 - val_accuracy: 0.6102
Epoch 6/11
2/2 [==============================] - 14s 11s/step - loss: 1.2152 - accuracy: 0.5254 - val_loss: 1.1702 - val_accuracy: 0.6102
Epoch 7/11
2/2 [==============================] - 13s 9s/step - loss: 1.2037 - accuracy: 0.5254 - val_loss: 1.1627 - val_accuracy: 0.6102
Epoch 8/11
2/2 [=================

### Results

## Densenet

In [51]:
width = train_images.shape[1]
height = train_images.shape[2]
bands = 3

densenet = tf.keras.applications.densenet.DenseNet121(
    include_top=False,
    weights='imagenet',
    input_shape=(width, height, bands),
    pooling=None,
    classifier_activation='softmax'
)

In [ ]:
densenet.trainable = False


inputs = keras.Input(shape=(width, height, bands))
x = densenet(inputs, training=False)

x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(5)(x)
densenet_model = keras.Model(inputs, outputs)

In [ ]:
densenet_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

densenet_model.fit(X_train, y_train, epochs=11, 
                    validation_data=(X_test, y_test))

## Mobilenet

In [1]:
mobilenet = tf.keras.applications.mobilenet.MobileNet(
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    pooling=None,
    classifier_activation='softmax',
)

NameError: name 'tf' is not defined

In [ ]:
mobilenet.trainable = False


inputs = keras.Input(shape=(width, height, bands))
x = mobilenet(inputs, training=False)

x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(5)(x)
mobilenet_model = keras.Model(inputs, outputs)

In [ ]:
mobilenet_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

mobilenet_model.fit(X_train, y_train, epochs=11, 
                    validation_data=(X_test, y_test))